In [1]:
import pandas as pd
import numpy as np
import numpy as np
from scipy.signal import fftconvolve
from concurrent.futures import ThreadPoolExecutor
import numpy.fft as fft
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_parquet('../../data/features/dsp_real.parquet', engine='pyarrow')  # You can use 'fastparquet' as the engine
data

,0,1,2,3,4,5,6,7,8,9,...,29578,29579,29580,29581,29582,29583,29584,29585,Target,Test
0,853.0,348.972321,868.135254,304.637085,248.844421,313.216644,473.186005,496.149414,393.626892,192.783401,...,393.626892,496.149414,473.186005,313.216644,248.844421,304.637085,868.135254,348.972321,XBB.1.5,0
1,846.0,365.208160,874.220520,291.514740,261.239227,297.506012,502.425934,485.041687,416.813049,190.762634,...,416.813049,485.041687,502.425934,297.506012,261.239227,291.514740,874.220520,365.208160,AY.116,0
2,839.0,371.940918,864.892334,272.480682,266.358032,298.774811,490.534485,500.755951,407.430328,191.879364,...,407.430328,500.755951,490.534485,298.774811,266.358032,272.480682,864.892334,371.940918,B.1.525,1
3,847.0,421.398254,858.878052,249.573563,282.337341,310.449371,575.844299,424.742188,434.341309,208.388718,...,434.341309,424.742188,575.844299,310.449371,282.337341,249.573563,858.878052,421.398254,B.1.1.57,0
4,813.0,375.336395,895.354187,333.891266,177.151901,368.090271,404.249878,557.433838,284.134979,179.973099,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,BQ.1.1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47317,854.0,350.274597,867.476501,298.539642,256.841003,307.958649,476.514435,495.816040,397.213562,183.639114,...,397.213562,495.816040,476.514435,307.958649,256.841003,298.539642,867.476501,350.274597,CP.5,1
47322,820.5,383.432617,860.243225,266.840759,285.087189,287.815918,494.647278,508.266266,455.361603,209.637680,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,BE.7,1
47324,803.5,397.387054,862.049805,262.277710,288.772064,289.236267,493.074738,506.503113,445.869202,210.246704,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,BE.7,1
47344,827.0,400.991516,842.978027,254.147156,304.774139,272.095764,525.074524,478.432129,458.260590,165.577026,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,BA.1,1


In [3]:
def random_undersample(data_df, max_samples_per_class=5, random_state=42):
    
    data_df = pd.DataFrame(data)
    
    undersampled_data = []

    for class_value, group in data_df.groupby('Target'):
        if len(group) > max_samples_per_class:
            undersampled_group = group.sample(n=max_samples_per_class, random_state=random_state)
        else:
            undersampled_group = group
        undersampled_data.append(undersampled_group)

    undersampled_data_df = pd.concat(undersampled_data)
    undersampled_data_df = undersampled_data_df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    return undersampled_data_df

In [4]:
pair_data = random_undersample(data[data["Test"] == 0])

In [5]:
# Ensure data_array and pair_data_array are NumPy arrays
data_array = data.drop(columns=["Test", "Target"]).to_numpy()
pair_data_array = pair_data.drop(columns=["Test", "Target"]).to_numpy()

# Standardize the data and pair_data arrays
data_std = (data_array - np.mean(data_array, axis=1, keepdims=True)) / np.std(data_array, axis=1, keepdims=True)
pair_data_std = (pair_data_array - np.mean(pair_data_array, axis=1, keepdims=True)) / np.std(pair_data_array, axis=1, keepdims=True)

# Compute the dot product between standardized data and pair_data (transposed)
# Since we're dealing with standardized data, the dot product gives us the sum of the products of standardized scores
dot_product = np.dot(data_std, pair_data_std.T)

# Divide by the number of observations to get the Pearson correlation coefficients
# Adjust n-1 for Pearson's correlation if needed, depending on your definition of standard deviation (np.std by default uses n)
n = data_array.shape[1]
correlation_matrix = dot_product / n

In [6]:
distance_matrix = (1 - correlation_matrix) / 2

In [7]:
distance_matrix = pd.DataFrame(distance_matrix)

In [8]:
distance_matrix["Target"] = data["Target"].to_list()
distance_matrix["Test"] = data["Test"].to_list()

In [9]:
distance_matrix.to_parquet('../../data/features/dsp_real_dist.parquet', engine='pyarrow')

/opt/conda/lib/python3.10/site-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)
